In [ ]:
!python3 -m pip install -U git+https://github.com/facebookresearch/audiocraft#egg=audiocraft
# !python3 -m pip install -U audiocraft

In [ ]:
from audiocraft.models import musicgen
from audiocraft.utils.notebook import display_audio
from audiocraft.data.audio import audio_write
import torch
import scipy

In [ ]:
# MusicGenの学習済みモデルをロード
model_music = musicgen.MusicGen.get_pretrained('medium', device='cuda')

# 生成する音楽の時間指定
model_music.set_generation_params(duration=20)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install openmeteo_requests
!pip install requests_cache
!pip install retry_requests

In [ ]:
!pip install openai

In [ ]:
# 天気予報APIで天気と外の気温，湿度を取得
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 34.69,
	"longitude": 135.1955,
	"current": ["temperature_2m", "relative_humidity_2m", "weather_code"],
	"timezone": "Asia/Tokyo",
	"forecast_days": 1
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]

# Current values. The order of variables needs to be the same as requested.
current = response.Current()
current_temperature_2m = int(current.Variables(0).Value())
current_relative_humidity_2m = int(current.Variables(1).Value())
current_weather_code = current.Variables(2).Value()

if current_weather_code == 0:
    current_weather = 'clear sky'
elif current_weather_code == 1 or current_weather_code == 2 or current_weather_code == 3:
    current_weather = 'partly cloudy'
elif current_weather_code == 45 or current_weather_code == 48:
    current_weather = 'fog'
elif current_weather_code == 51 or current_weather_code == 53 or current_weather_code == 55:
    current_weather = 'drizzle'
elif current_weather_code == 56 or current_weather_code == 57:
    current_weather = 'freezing drizzle'
elif current_weather_code == 61 or current_weather_code == 63 or current_weather_code == 65:
    current_weather = 'rain'
elif current_weather_code == 66 or current_weather_code == 67:
    current_weather = 'freezing rain'
elif current_weather_code == 71 or current_weather_code == 73 or current_weather_code == 75:
    current_weather = 'snow fall'
elif current_weather_code == 77:
    current_weather = 'snow grains'
elif current_weather_code == 80 or current_weather_code == 81 or current_weather_code == 82:
    current_weather = 'rain showers'
elif current_weather_code == 85 or current_weather_code == 86:
    current_weather = 'snow showes'
elif current_weather_code == 95:
    current_weather = 'thunderstorm'
elif current_weather_code == 96 or current_weather_code == 99:
    current_weather = 'thunderstorm with hail'
else:
    current_weather = ""


def get_weather():
    weather = f'{current_weather}, the outside temperature is {current_temperature_2m}°C with {current_relative_humidity_2m}% humidity'
    return weather


In [ ]:
!pip install torch torchvision opencv-python

In [ ]:
import torch
import torchvision
model_detect = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

In [ ]:
# 物体検知用のラベル
LABELS = [
        "unlabeled",
        "person",
        "bicycle",
        "car",
        "motorcycle",
        "airplane",
        "bus",
        "train",
        "truck",
        "boat",
        "traffic light",
        "fire hydrant",
        "street sign",
        "stop sign",
        "parking meter",
        "bench",
        "bird",
        "cat",
        "dog",
        "horse",
        "sheep",
        "cow",
        "elephant",
        "bear",
        "zebra",
        "giraffe",
        "hat",
        "backpack",
        "umbrella",
        "shoe",
        "eye glasses",
        "handbag",
        "tie",
        "suitcase",
        "frisbee",
        "skis",
        "snowboard",
        "sports ball",
        "kite",
        "baseball bat",
        "baseball glove",
        "skateboard",
        "surfboard",
        "tennis racket",
        "bottle",
        "plate",
        "wine glass",
        "cup",
        "fork",
        "knife",
        "spoon",
        "bowl",
        "banana",
        "apple",
        "sandwich",
        "orange",
        "broccoli",
        "carrot",
        "hot dog",
        "pizza",
        "donut",
        "cake",
        "chair",
        "couch",
        "potted plant",
        "bed",
        "mirror",
        "dining table",
        "window",
        "desk",
        "toilet",
        "door",
        "tv",
        "laptop",
        "mouse",
        "remote",
        "keyboard",
        "cell phone",
        "microwave",
        "oven",
        "toaster",
        "sink",
        "refrigerator",
        "blender",
        "book",
        "clock",
        "vase",
        "scissors",
        "teddy bear",
        "hair drier",
        "toothbrush",
    ]

In [ ]:
# 写真撮影
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      // await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
# 人数カウント
import PIL.Image

def count_people(path_photo):
  labels = LABELS
  image = PIL.Image.open(path_photo)
  image_tensor = torchvision.transforms.functional.to_tensor(image)

  model_detect_eval = model_detect.eval()
  output = model_detect_eval([image_tensor])[0]

  num_people = 0
  for label, score in zip(output['labels'], output['scores']):
    if score > 0.5 and labels[label] == 'person':
      num_people += 1
  print(f'number of people:{num_people}')

  return num_people


In [ ]:
# chatGPT API
from openai import OpenAI

client = OpenAI(api_key='')

def get_chatgpt_response(message):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        max_tokens=30,
        presence_penalty=1.0,
        frequency_penalty=1.0,
        temperature=1,
        messages=[
            {"role": "user", "content": message}
        ]
    )

    response = completion.choices[0].message.content
    print(response)

    return response

In [ ]:
# 認証のためのコード
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
## スプレッドシートを開く（名前から）
def get_sensor_data():
  filename = "sensor_data"
  ss = gc.open(filename)

  # 最終行の取得
  st = ss.get_worksheet(0)
  list_of_lists = st.get_all_values()
  latest = len(list_of_lists)

  print(f'Date_Master:{st.cell(latest, 1).value}')
  print(f'Date:{st.cell(latest, 2).value}')

  # 室内気温と室内湿度取得
  print(f'latest indoor temperature:{st.cell(latest, 3).value}')
  temperature_indoor = st.cell(latest, 3).value

  print(f'latest indoor humidity:{st.cell(latest, 4).value}')
  humidity_indoor = st.cell(latest, 4).value

  # 明るさを取得
  print(f'latest brightness:{st.cell(latest, 5).value}')
  brightness = st.cell(latest, 5).value

  print(f'Date_End:{st.cell(latest, 6).value}')

  return temperature_indoor, humidity_indoor, brightness

In [ ]:
from math import e
from pydub import AudioSegment
import soundfile as sf
import datetime
import shutil
from pydub.playback import play
import time
import cv2

# while 1:
# 現在時刻と天気を取得
dt_now = datetime.datetime.now()+ datetime.timedelta(hours=9)
dt_now_str = dt_now.strftime('%Y %m/%d %H:%M')
weather = get_weather()

# センサデータを取得
temperature_indoor, humidity_indoor, brightness = get_sensor_data()

# 写真撮影
try:
  filename = take_photo()
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

# 人数カウント
num_people = count_people('./photo.jpg')


# chatGPTにテキストを入力
message = f'{dt_now_str}, {weather}, indoor temperature is {temperature_indoor}°C with {humidity_indoor}% humidity, the brightness in the room is {brightness} lux, {num_people} people, \nDescribe in 20 words the musical atmosphere suitable for this. '

print(message)

response = get_chatgpt_response(message)
# response = message

print(response)

for i in range(3):   # 次のセンサ情報をchatGPTに入力するまで，同じ入力内容で何回か音楽を生成する
  # 音楽生成
  res = model_music.generate([response],
      progress=True)
  display_audio(res, model_music.sample_rate)


  # ファイル名をここで設定
  file_name = f'musicgen_out_{dt_now}_{i+1}'

  audio_data_cpu = res[0].cpu().numpy()
  audio_data_reshaped = audio_data_cpu.reshape(-1, 1)

  # WAV形式で一時的に保存
  temp_wav_path = '/content/' + file_name +'.wav'
  sf.write(temp_wav_path, audio_data_reshaped, model_music.sample_rate)

  # MP3形式に変換
  audio_segment = AudioSegment.from_wav(temp_wav_path)
  mp3_path = '/content/' + file_name +'.mp3'
  audio_segment.export(mp3_path, format="mp3")

  # MP3形式で保存
  destination_folder_path = '/content/drive/MyDrive/generate_music/playlist'
  shutil.copy(mp3_path, destination_folder_path)
